In [46]:
import pandas as pd
import numpy as np
from pathlib import Path
import json

DATA_DIR = Path("datasets")
EVENTS_DIR = DATA_DIR.joinpath("events")
TRANSFERS_DIR = DATA_DIR.joinpath("transfers")

# Explore the [events](https://www.kaggle.com/datasets/secareanualin/football-events/data) dataset

In [47]:
events_df = pd.read_csv(EVENTS_DIR.joinpath("events.csv"))
game_info_df = pd.read_csv(EVENTS_DIR.joinpath("ginf.csv"))
dictionary = json.load(open(EVENTS_DIR.joinpath("dictionary.json")))

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/events/events.csv'

In [ ]:
full_df = pd.merge(events_df, game_info_df, on='id_odsp', how='inner').drop("id_odsp", axis=1).drop("id_event", axis=1).drop("sort_order", axis=1).drop("adv_stats", axis=1)
for name in dictionary.keys():
    full_df = full_df.replace({name: { kv["id"] : kv["name"] for kv in dictionary[name]}})
full_df

In [48]:
full_df[full_df.is_goal == 1].player.value_counts()

AttributeError: 'DataFrame' object has no attribute 'is_goal'

In [49]:
frame = {'Home': game_info_df[["ht"]].value_counts(),
         'Away': game_info_df[["at"]].value_counts()}
games_df = pd.DataFrame(frame)
games_df['Total'] = games_df.Home + games_df.Away
games_df.value_counts("Total").head()

NameError: name 'game_info_df' is not defined

In [50]:
game_info_df[["season", "country"]].value_counts().head()

NameError: name 'game_info_df' is not defined

In [51]:
full_df[full_df.is_goal == 1].location.dropna().value_counts()

AttributeError: 'DataFrame' object has no attribute 'is_goal'

In [52]:
full_df[full_df.is_goal == 1].shot_place.value_counts()

AttributeError: 'DataFrame' object has no attribute 'is_goal'

In [53]:
full_df[full_df.is_goal == 1]

AttributeError: 'DataFrame' object has no attribute 'is_goal'

In [54]:
all_attempts = full_df[full_df.event_type == "Attempt"]
goals_by_shot_place = all_attempts[all_attempts.is_goal == 1].groupby("shot_place").shot_place.size()
attempts_by_shot_place = all_attempts.groupby("shot_place").shot_place.size()
goal_rates = pd.concat([goals_by_shot_place, attempts_by_shot_place], axis=1, join="inner")
goal_rates.columns = ['goals','attempts']
goal_rates['ratio'] = goal_rates["goals"] / goal_rates["attempts"]
goal_rates

AttributeError: 'DataFrame' object has no attribute 'event_type'

## Notes
- The 2017 season is not complete
- The position data for each event is not precise
- Only 1/3 events have a location, but moments of play such as goals often do
- Most goals are scored in the bottom corners, but most attempts are made in the centre

# Explore the [transfers](https://github.com/d2ski/football-transfers-data) dataset


In [55]:
full_df = pd.read_csv(TRANSFERS_DIR.joinpath("transfers.csv"))
full_df

,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_id,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id
0,GB1,2009,s,985,Manchester United,England,in,33544,Antonio Valencia,23.0,...,1071,Wigan Athletic,England,18800000.0,NaN,False,False,False,False,310832
1,GB1,2009,s,985,Manchester United,England,in,62049,Mame Diouf,21.0,...,687,Molde FK,Norway,4500000.0,1600000.0,False,False,False,False,319841
2,GB1,2009,s,985,Manchester United,England,in,43261,Gabriel Obertan,20.0,...,40,FC Girondins Bordeaux,France,4000000.0,400000.0,False,False,False,False,315185
3,GB1,2009,s,985,Manchester United,England,in,1397,Michael Owen,29.0,...,762,Newcastle United,England,0.0,NaN,True,False,False,False,306421
4,GB1,2009,s,985,Manchester United,England,in,73538,Scott Moffatt,18.0,...,5242,Manchester United U18,England,NaN,NaN,False,False,False,False,339015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70001,NL1,2021,w,306,SC Heerenveen,Netherlands,in,257808,Runar Espejord,25.0,...,1293,Tromsø IL,Norway,NaN,500000.0,False,True,True,False,3071862
70002,NL1,2021,w,306,SC Heerenveen,Netherlands,in,580142,Joaquín Fernández,22.0,...,37535,Montevideo City Torque,Uruguay,NaN,100000.0,False,True,True,False,3268245
70003,NL1,2021,w,468,Sparta Rotterdam,Netherlands,in,340353,Maduka Okoye,22.0,...,1010,Watford FC,England,NaN,1000000.0,False,True,False,False,3619917
70004,NL1,2021,w,468,Sparta Rotterdam,Netherlands,left,340353,Maduka Okoye,22.0,...,1010,Watford FC,England,7000000.0,1000000.0,False,False,False,False,3619916


In [56]:
transfers_in_df = full_df[(full_df['dir'] == 'in') & (full_df['is_retired'] == False) & (full_df['is_loan_end'] == False)]
transfers_in_df

,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_id,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id
0,GB1,2009,s,985,Manchester United,England,in,33544,Antonio Valencia,23.0,...,1071,Wigan Athletic,England,18800000.0,NaN,False,False,False,False,310832
1,GB1,2009,s,985,Manchester United,England,in,62049,Mame Diouf,21.0,...,687,Molde FK,Norway,4500000.0,1600000.0,False,False,False,False,319841
2,GB1,2009,s,985,Manchester United,England,in,43261,Gabriel Obertan,20.0,...,40,FC Girondins Bordeaux,France,4000000.0,400000.0,False,False,False,False,315185
3,GB1,2009,s,985,Manchester United,England,in,1397,Michael Owen,29.0,...,762,Newcastle United,England,0.0,NaN,True,False,False,False,306421
4,GB1,2009,s,985,Manchester United,England,in,73538,Scott Moffatt,18.0,...,5242,Manchester United U18,England,NaN,NaN,False,False,False,False,339015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69983,NL1,2021,s,403,Willem II Tilburg,Netherlands,in,584452,Nikolaos Michelis,20.0,...,10958,Milan Primavera,Italy,NaN,400000.0,False,True,False,False,3438791
69984,NL1,2021,s,403,Willem II Tilburg,Netherlands,in,420219,Vincent Schippers,20.0,...,9005,Willem II Tilburg U21,Netherlands,NaN,75000.0,False,False,False,False,3414293
70000,NL1,2021,w,385,Fortuna Sittard,Netherlands,in,140032,Andreas Samaris,32.0,...,515,Without Club,Without Club,NaN,1500000.0,False,False,False,False,3633188
70003,NL1,2021,w,468,Sparta Rotterdam,Netherlands,in,340353,Maduka Okoye,22.0,...,1010,Watford FC,England,NaN,1000000.0,False,True,False,False,3619917


In [57]:
seasonal_transfers = transfers_in_df.season.value_counts()
display(seasonal_transfers)
print("Average transfers per season: {mean}".format(mean=seasonal_transfers.sum() / seasonal_transfers.size))

season
2015    1951
2013    1920
2014    1909
2019    1895
2016    1886
2018    1883
2011    1861
2017    1835
2012    1814
2009    1665
2010    1660
2020    1643
2021    1291
Name: count, dtype: int64

Average transfers per season: 1785.6153846153845


In [58]:
transfers_per_club = transfers_in_df[(transfers_in_df['is_loan'] == False)].team_name.value_counts()
display(transfers_per_club)
print("Average transfers per club and season: {mean}".format(mean=transfers_per_club.sum() / (transfers_per_club.size * seasonal_transfers.size)))

team_name
Genoa CFC             271
Parma FC              255
Udinese Calcio        183
Vitória Setúbal FC    181
SL Benfica            177
                     ... 
GFC Ajaccio             7
Clermont Foot 63        7
Brentford FC            6
CD Tenerife             4
FC St. Pauli            2
Name: count, Length: 242, dtype: int64

Average transfers per club and season: 5.715511760966306


In [59]:
transfers_in_df[(transfers_in_df['transfer_fee_amnt'] > 220000000)]

,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_id,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id
19873,IT1,2009,s,46,FC Internazionale,Italy,in,83404,Matteo Lombardo,24.0,...,4550,Legnano AC,Italy,500000000.0,NaN,False,False,False,False,382457
20442,IT1,2009,s,1627,Calcio Catania,Italy,in,41312,Simone Pesce,27.0,...,408,Ascoli Calcio 1898,Italy,500000000.0,200000.0,False,False,False,False,338638
20595,IT1,2009,s,1390,Cagliari Calcio,Italy,in,35719,Andrea Cocco,24.0,...,24361,Alghero 1945,Italy,500000000.0,75000.0,False,False,False,False,434798
20618,IT1,2009,s,1025,Bologna FC 1909,Italy,in,139326,Giuseppe Nazzani,19.0,...,10643,AC Bellaria Igea Marina,Italy,500000000.0,NaN,False,False,False,False,399981
21534,IT1,2010,s,252,Genoa CFC,Italy,in,62270,Danilo Russo,23.0,...,2655,Vicenza Calcio,Italy,500000000.0,100000.0,False,False,False,False,590150
21719,IT1,2010,s,12,AS Roma,Italy,in,41310,Stefano Guberti,26.0,...,1038,UC Sampdoria,Italy,500000000.0,50000.0,False,False,False,False,590004
22049,IT1,2011,s,6692,Novara Calcio 1908,Italy,in,28935,Paolo Dellafiore,26.0,...,130,Parma FC,Italy,500000000.0,25000.0,False,False,False,False,616725
22087,IT1,2011,s,6195,SSC Napoli,Italy,in,21690,Daniele Mannini,27.0,...,1038,UC Sampdoria,Italy,500000000.0,NaN,False,False,False,False,590138
22731,IT1,2011,s,252,Genoa CFC,Italy,in,91914,Davide Bertoncini,21.0,...,1408,Piacenza FC,Italy,550000000.0,300000.0,False,False,False,False,735935
22923,IT1,2011,s,12,AS Roma,Italy,in,24001,Gianluca Curci,25.0,...,1038,UC Sampdoria,Italy,500000000.0,NaN,False,False,False,False,590007


In [60]:
player_transfers = transfers_in_df.player_id.value_counts()
display(player_transfers)
display(transfers_in_df[(transfers_in_df['player_id'] == 74297)])
print("Average transfers (including loans) per player and season: {mean}".format(mean=player_transfers.sum() / (player_transfers.size * seasonal_transfers.size)))


player_id
74297     13
16922     12
59340     11
37403     11
54935     11
          ..
17443      1
42033      1
692        1
61003      1
420219     1
Name: count, Length: 12748, dtype: int64

,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_id,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id
1657,GB1,2010,w,931,Fulham FC,England,in,74297,Gaël Kakuta,19.0,...,631,Chelsea FC,England,NaN,8000000.0,False,True,False,False,531885
2324,GB1,2011,s,355,Bolton Wanderers,England,in,74297,Gaël Kakuta,20.0,...,631,Chelsea FC,England,NaN,8000000.0,False,True,False,False,630276
14414,ES1,2014,s,367,Rayo Vallecano,Spain,in,74297,Gaël Kakuta,23.0,...,631,Chelsea FC,England,NaN,8000000.0,False,True,False,False,1100082
14945,ES1,2015,s,368,Sevilla FC,Spain,in,74297,Gaël Kakuta,24.0,...,631,Chelsea FC,England,6000000.0,8000000.0,False,False,False,False,1259969
16255,ES1,2016,w,897,Deportivo de La Coruña,Spain,in,74297,Gaël Kakuta,25.0,...,36630,Hebei China Fortune,China,NaN,8000000.0,False,True,False,False,1676521
17589,ES1,2018,s,367,Rayo Vallecano,Spain,in,74297,Gaël Kakuta,27.0,...,36630,Hebei China Fortune,China,2000000.0,8000000.0,False,False,False,False,2174554
26063,IT1,2013,w,398,SS Lazio,Italy,in,74297,Gaël Kakuta,22.0,...,631,Chelsea FC,England,NaN,8000000.0,False,True,False,False,1012276
44645,FR1,2011,w,2969,Dijon FCO,France,in,74297,Gaël Kakuta,20.0,...,631,Chelsea FC,England,NaN,8000000.0,False,True,False,False,670312
47816,FR1,2017,s,1416,Amiens SC,France,in,74297,Gaël Kakuta,26.0,...,36630,Hebei China Fortune,China,NaN,8000000.0,False,True,False,False,1877834
49222,FR1,2019,s,1416,Amiens SC,France,in,74297,Gaël Kakuta,28.0,...,367,Rayo Vallecano,Spain,3000000.0,8000000.0,False,False,False,False,2597823


Average transfers (including loans) per player and season: 0.1400702372619536


In [61]:

player_transfers_wo_loans = transfers_in_df[(transfers_in_df['is_loan'] == False)].player_id.value_counts()
display(player_transfers_wo_loans)
display(transfers_in_df[(transfers_in_df['player_id'] == 16922)])
print("Average transfers (excluding loans) per player and season: {mean}".format(mean=player_transfers_wo_loans.sum() / (player_transfers_wo_loans.size * seasonal_transfers.size)))


player_id
16922     10
37403      8
162961     8
129990     7
96754      7
          ..
421775     1
457236     1
339566     1
331514     1
420219     1
Name: count, Length: 11894, dtype: int64

,league,season,window,team_id,team_name,team_country,dir,player_id,player_name,player_age,...,counter_team_id,counter_team_name,counter_team_country,transfer_fee_amnt,market_val_amnt,is_free,is_loan,is_loan_end,is_retired,transfer_id
370,GB1,2009,s,1020,Portsmouth FC,England,in,16922,Kevin-Prince Boateng,22.0,...,148,Tottenham Hotspur,England,4500000.0,2000000.0,False,False,False,False,337574
16129,ES1,2016,s,472,UD Las Palmas,Spain,in,16922,Kevin-Prince Boateng,29.0,...,5,AC Milan,Italy,0.0,2000000.0,True,False,False,False,1483950
17705,ES1,2018,w,131,FC Barcelona,Spain,in,16922,Kevin-Prince Boateng,31.0,...,6574,US Sassuolo,Italy,1000000.0,2000000.0,False,True,False,False,2377385
21329,IT1,2010,s,5,AC Milan,Italy,in,16922,Kevin-Prince Boateng,23.0,...,252,Genoa CFC,Italy,3000000.0,2000000.0,False,True,False,False,467628
21523,IT1,2010,s,252,Genoa CFC,Italy,in,16922,Kevin-Prince Boateng,23.0,...,1020,Portsmouth FC,England,5700000.0,2000000.0,False,False,False,False,467627
22433,IT1,2011,s,5,AC Milan,Italy,in,16922,Kevin-Prince Boateng,24.0,...,252,Genoa CFC,Italy,10500000.0,2000000.0,False,False,False,False,572326
28936,IT1,2015,w,5,AC Milan,Italy,in,16922,Kevin-Prince Boateng,28.0,...,515,Without Club,Without Club,NaN,2000000.0,False,False,False,False,1396248
32409,IT1,2018,s,6574,US Sassuolo,Italy,in,16922,Kevin-Prince Boateng,31.0,...,24,Eintracht Frankfurt,Germany,0.0,2000000.0,True,False,False,False,2161758
33061,IT1,2019,s,430,ACF Fiorentina,Italy,in,16922,Kevin-Prince Boateng,32.0,...,6574,US Sassuolo,Italy,1500000.0,2000000.0,False,False,False,False,2583138
38133,L1,2013,s,33,FC Schalke 04,Germany,in,16922,Kevin-Prince Boateng,26.0,...,5,AC Milan,Italy,10000000.0,2000000.0,False,False,False,False,940485


Average transfers (excluding loans) per player and season: 0.11629004928147353


## Notes
- The data contains two rows for each transfer between clubs. One explaining the "receiving" club and one explaining the "giving" club. The direction is given in the 'dir' column. Player retirements are also included in the dataset.
- Average transfers (including loans) per season is 1785,62 for the seasons 2009 - 2021. Since the transfers do not have dates it's hard to check if all the years are complete.
- The club acquiring most players (excluding loans and retirements) is Genoa CFC acquiring a total of 271 players between 2009 - 2021. The average amount of acquiring (excluding loans and retirements) per club and season is 5,72.
- There is some error in the transfer fee amount data. According to other sources the largest transfer fee amount in history is when Neymar was sold from FC Barcelona to Paris SG for 220 million Euro. In this data set there are 13 transfers in Serie A with reported transfer fee amounts up to 550 million Euro.
- The player with most transfers (including loans) is Gaël Kakuta from France who did 13 transfers between 2009 - 2021. The last transfer was not between clubs but going from a loan to a contract within RC Lens. The average amount of transfers (including loans) per player and season is 0,14.
- The player with most transfers (excluding loans) is Kevin-Prince Boateng who did 10 transfers between 2009 - 2021. The average amount of transfers (excluding loans) per player and season is 0,12.
